## Define dHCP pipeline

In [1]:
def dhcp_tractography_pipeline(subject, session, aws_access_key, aws_secret_key, dilation_size=12, streamline_count=2000000, local_env=False):
    """
    For each `subject`, `session` pair run tractography pipeline.
    
    0. Perpare local environment
    1. Generate isotropic DWI files
    2. Realign DWI to anatomy using rigid transformation
    3. Estimate CSD response function
    4. Reassign ribbon values
    5. Generate five-tissue-type (5TT) segmented tissue image
    6. Tractography
    7. Create BIDS derivatives
    8. Upload to s3
    
    Parameters
    ----------
    subject : string
    session : string
    aws_access_key : string
    aws_secret_key : string
    streamline_count : int
    local_env : boolean
    """
    
    import subprocess
    import os
    from os.path import exists, join
    import s3fs
    import json
    
    fs = s3fs.S3FileSystem(
        key=aws_access_key,
        secret=aws_secret_key
    )
    
    def print_quiet(string):
        """
        mrtrix doesn't provide a way to conviently supress progress messages without
        also supressing informational messages. 
        
        filter stderr/stdout before printing

        """
        import re
        
        # remove any line that looks like a progress message
        string = re.sub(r'.*\[[0-9 ]{3}\%\].*\n?', '', string)

        # remove empty lines
        string = re.sub(r'^\n$', '', string)
        
        # remove any leading or trailing whitespace
        string = string.strip()
        
        if string != "":
            print(string, flush=True)
        
    
    ###############################################################################
    # Step 0: Prepare local environment
    ###############################################################################
    
    print('Step 0: Prepare local environment', flush=True)
    
    os.makedirs(join('input', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    
    def download_dhcp_files(subject, session, fs):
        """
        Download anatomy and DWI files for tractography
        
        Parameters
        ----------
        subject : string
        session : string
        aws_access_key : string
        aws_secret_key : string
        """
        from os.path import exists, join
    
        anat_files = [
            f'sub-{subject}_ses-{session}_desc-drawem87_space-T2w_dseg.nii.gz',
            f'sub-{subject}_ses-{session}_desc-drawem9_space-T2w_dseg.nii.gz',
            f'sub-{subject}_ses-{session}_desc-restore_T2w.nii.gz',
            f'sub-{subject}_ses-{session}_desc-ribbon_space-T2w_dseg.nii.gz'
        ]
        
        for anat_file in anat_files:
            if not exists(join('input', f'sub-{subject}', f'ses-{session}', anat_file)):
                print('downloading:', anat_file, flush=True)
                fs.get(
                    (
                        f'dhcp-afq/dhcp_anat_pipeline/sub-{subject}/ses-{session}/anat/'
                        f'{anat_file}'
                    ),
                    join('input', f'sub-{subject}', f'ses-{session}', anat_file)
                )

        dwi_files = [
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.bval',
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.bvec',
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.nii.gz',
            f'sub-{subject}_ses-{session}_desc-preproc_space-dwi_brainmask.nii.gz'
        ]

        for dwi_file in dwi_files:
            if not exists(join('input', f'sub-{subject}', f'ses-{session}', dwi_file)):
                print('downloading:', dwi_file, flush=True)
                fs.get(
                    (
                        f's3://dhcp-afq/dhcp_dmri_pipeline/sub-{subject}/ses-{session}/dwi/'
                        f'{dwi_file}'
                    ),
                    join('input', f'sub-{subject}', f'ses-{session}', dwi_file)
                )
    
    download_dhcp_files(subject, session, fs)
    
    ###############################################################################
    # Step 1: Generate isotropic DWI files
    ###############################################################################
    
    print('Step 1: Generate isotropic DWI files', flush=True)
    
    def reslice_dwi(img_filename, new_zooms=(1.5, 1.5, 1.5)):
        """
        Creates an isotropic dwi file
        
        In dHCP dataset dwi images have voxel size [1.172 1.172 1.5]
        therefore by default scaling to 1.5
        
        Parameters
        ----------
        subject : string
        session : string
        """
        import os.path as op
        import nibabel as nib
        from dipy.align.reslice import reslice

        img = nib.load(img_filename)
        img_data = img.get_fdata()
        img_affine = img.affine
        img_zooms = img.header.get_zooms()[:3]
        new_data, new_affine = reslice(img_data, img_affine, img_zooms, new_zooms)
        new_img = nib.Nifti1Image(new_data, new_affine)

        # use original image name as basis for the resliced image name
        img_basename = op.basename(img_filename)
        desc_index = img_basename.rindex('_')
        img_name_start = img_basename[:desc_index]
        img_name_end = img_basename[desc_index:]
        new_img_filename = op.join('output', f'sub-{subject}', f'ses-{session}', f'{img_name_start}_resliced{img_name_end}')
        
        print('saving resliced dwi:', new_img_filename, flush=True)
        nib.save(new_img, new_img_filename)
    
    
    # dwi
    reslice_dwi(join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_dwi.nii.gz'))

    # brainmask
    reslice_dwi(join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_space-dwi_brainmask.nii.gz'))

    ###############################################################################
    # Step 2: Realign DWI to anatomy using rigid transformation
    ###############################################################################
    
    print('Step 2: Realign DWI to anatomy using rigid transformation', flush=True)

    # convert NIFTI to MIF for MRtrix and generate b file
    # doing this because MRtrix `works better` with these formats
    mrconvert = subprocess.run(
        [
            'mrconvert',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.nii.gz',
            '-fslgrad',
            f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_dwi.bvec',
            f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_dwi.bval',
            '-export_grad_mrtrix',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrconvert.stdout)
    print_quiet(mrconvert.stderr)
    
    # generate b0 from dwi
    dwiextract = subprocess.run(
        [
            'dwiextract',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            '-grad',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-',
            '-bzero'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    
    # don't bother printing out since piping to mrmath
    # print(dwiextract.stdout)
    # print(dwiextract.stderr)
    
    # pipe dwiextract to mrmath
    mrmath = subprocess.run(
        [
            'mrmath',
            '-',
            'mean',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-resliced_b0.mif',
            '-axis',
            '3'
        ],
        input = dwiextract.stdout,
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrmath.stdout)
    print_quiet(mrmath.stderr)
        
    # registered dwi to anat and extract transform
    mrregister = subprocess.run(
        [
            'mrregister',
            '-type',
            'rigid',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-resliced_b0.mif',
            f'input/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-restore_T2w.nii.gz',
            '-transformed',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-resliced_aligned_b0.mif',
            '-rigid',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrregister.stdout)
    print_quiet(mrregister.stderr)
    
    # apply transform to dwi
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            '-grad',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-export_grad_mrtrix',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b',
            '-linear',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # export a NIFTI version (for pyafq) with FSL
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.nii.gz',
            '-grad',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-export_grad_fsl',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bvec',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bval',
            '-linear',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # apply transform to brainmask
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space-dwi_resliced_brainmask.nii.gz',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-linear',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # export a NIFTI version (for pyafq)
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space-dwi_resliced_brainmask.nii.gz',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz',
            '-linear',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    ###############################################################################
    # Step 3: Estimate CSD response function
    ###############################################################################
    
    print('Step 3: Estimate CSD response function', flush=True)
    
    # tissue response function
    dwi2response = subprocess.run(
        [
            'dwi2response',
            'dhollander',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm.txt',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-gm.txt',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csf.txt',
            '-mask',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-grad',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b',
            '-voxels',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-voxels.mif',
            '-fa',
            '0.1'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(dwi2response.stdout)
    print_quiet(dwi2response.stderr)
    
    # CSD estimates
    dwi2fod = subprocess.run(
        [
            'dwi2fod',
            'msmt_csd',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm.txt',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csf.txt',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csf.mif',
            '-mask',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-grad',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(dwi2fod.stdout)
    print_quiet(dwi2fod.stderr)
    
    # normalize
    mtnormalise = subprocess.run(
        [
            'mtnormalise',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm_norm.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csf.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csf_norm.mif',
            '-mask',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mtnormalise.stdout)
    print_quiet(mtnormalise.stderr)

    ###############################################################################
    # Step 4: Reassign ribbon values
    ###############################################################################
    
    print('Step 4: Reassign ribbon values', flush=True)

    def update_ribbon(subject, session, dilation_size):
        """
        - do not include ventricles (drawem9 5) as white matter (ribbon 41/2)
        - include brainstem (drawem9 8) as white matter (ribbon 41)
        - take corpus callosum (drawem87 48)
        - dilate the corpus callosum by `dilation_size`
        - label dilation as white matter (ribbon 41)
    
        ensure that there is sufficent white matter around the ventricles 
        for tracking, while attempting to keep the original gray-white
        matter boundary entact everywhere else

        TODO: probably could just use the Draw-EM 87
        Draw-EM 9 tissue segmentation definitions
        s3://dhcp-afq/dhcp_anat_pipeline/desc-drawem9_dseg.tsv
        
        Draw-EM 87 tissue segmentation definitions
        s3://dhcp-afq/dhcp_anat_pipeline/desc-drawem87_dseg.tsv
        
        Ribbon file uses FreeSurfer definitions
        https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT

        NOTE: hemisphere designation does not matter as mrtrix merges
        using right hemisphere labels

        Parameters
        ----------
        subject : string
        session : string
        dilation_size : int
        """

        from os.path import join
        import nibabel as nib
        import numpy as np
        from skimage.morphology import cube, dilation
        
        ribbon = nib.load(join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-ribbon_space-T2w_dseg.nii.gz'))
        ribbon_data = ribbon.get_fdata()
        
        # dilate the corpus callosum
        cc_mask_data = np.zeros(ribbon_data.shape)
        drawem87 = nib.load(join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-drawem87_space-T2w_dseg.nii.gz'))
        # corpus callosum=48
        cc_mask_data[np.where(drawem87.get_fdata()==48)] = 1
        dilate_cc_mask_data = dilation(cc_mask_data, cube(dilation_size))
        ribbon_data[np.where(dilate_cc_mask_data == 1)] = 41
    
        # add ventricles back in, in case got overwitten by cc dilation
        drawem9 = nib.load(join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-drawem9_space-T2w_dseg.nii.gz'))
        # ventricles=5 - label as lateral ventricle left=4 or right=43
        ribbon_data[np.where(drawem9.get_fdata() == 5)] = 43

        # brainstem=8 - label as white matter left=2 or right=41
        ribbon_data[np.where(drawem9.get_fdata() == 8)] = 41

        augmented_ribbon = nib.Nifti1Image(ribbon_data, ribbon.affine)
        ribbon_file = join('output', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-ribbon_space_agmntd-T2w_dseg.nii.gz')
        print('saving updated ribbon:', ribbon_file, flush=True)
        nib.save(augmented_ribbon, ribbon_file)
        return ribbon_file

    ribbon_file = update_ribbon(subject, session, dilation_size)

    ###############################################################################
    # Step 5. Generate five-tissue-type (5TT) segmented tissue image
    ###############################################################################
    
    print('Step 5. Generate five-tissue-type (5TT) segmented tissue image', flush=True)

    if local_env:
        os.environ["FREESURFER_HOME"] = '.'
    
    # 5tt
    fivettgen = subprocess.run(
        [
            '5ttgen',
            'freesurfer',
            '-lut',
            f'{os.environ["FREESURFER_HOME"]}/FreeSurferColorLUT.txt',
            ribbon_file,
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-tt5.mif',
            '-nocrop'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(fivettgen.stdout)
    print_quiet(fivettgen.stderr)
    
    # gmwmi
    fivett2gmwmi = subprocess.run(
        [
            '5tt2gmwmi',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-tt5.mif',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-tt5_gmwmi.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(fivett2gmwmi.stdout)
    print_quiet(fivett2gmwmi.stderr)
    
    ###############################################################################
    # Step 6. Tractography
    ###############################################################################
    
    print('Step 6. Tractography', flush=True)

    tckgen = subprocess.run(
        [
            'tckgen',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-wm_norm.mif',
            '-algo',
            'IFOD1',
            '-backtrack',
            '-crop_at_gmwmi',
            '-seed_gmwmi',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-tt5_gmwmi.mif',
            '-act',
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-tt5.mif',
            '-angle',
            '15',
            '-select',
            str(streamline_count),
            f'output/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_desc-csd_tractography.tck',
            '-cutoff',
            '0.05'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(tckgen.stdout)
    print_quiet(tckgen.stderr)
    
    ###############################################################################
    # Step 7. Create BIDS derivatives
    ###############################################################################
    
    print('Step 7. Create BIDS dataset_description', flush=True)
    
    dataset_description = {
        "Name" : "dHCP neonatal MRtrix derivatives",
        "PipelineDescription" : {
            "Name" : "dHCP neonatal MRtrix pipeline"
        },
        "BIDSVersion" : "1.4.0"
    }

    with open(join('output', 'dataset_description.json'), 'w') as f:
        json.dump(dataset_description, f)
    
    ###############################################################################
    # Step 8. Upload to s3
    ###############################################################################

    # if running locally may be processing multiple subjects, without cleaning
    # local directory in between and don't want to reupload same data multiple
    # times
    if not local_env:
        print('Step 8. Upload to s3', flush=True)
        
        # upload output folder to s3
        fs.put('output', 'dhcp-afq/mrtrix/', recursive=True)

## AWS Credentials

Get DIRECT credentials to download/upload to dhcp-afq S3 bucket

In [2]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()

aws_access_key = CP.get('default', 'AWS_ACCESS_KEY_ID')
aws_secret_key = CP.get('default', 'AWS_SECRET_ACCESS_KEY')

def attach_keys(arr):
    return [(e + (aws_access_key, aws_secret_key)) for e in arr]

In [20]:
def get_subject_session_pair(bucket_path):
    """
    find subject session tuples from s3 file system
    
    not all subjects have corresponding session data, and some have multiple
    
    there is no metadata that lists theses pairs, so traverse the bucket
    to identify
    
    Parameters
    ----------
    bucket_path : string
    
    Returns
    -------
    list of tuples containing subject_id and session_id
    """
    import s3fs
    fs = s3fs.S3FileSystem()

    subject_session_pairs = []

    for file in fs.ls(bucket_path):
        if fs.isdir(file):
            # directory bucket_path/sub-<subid>       
            subject = file.split('/')[-1].split('-')[-1]
            for file2 in fs.ls(file):
                if fs.isdir(file2):
                    # directory bucket_path/sub-<subid>/ses-<sesid>
                    session = file2.split('/')[-1].split('-')[-1]
                    subject_session_pairs.append((subject, session))
    
    return subject_session_pairs


def check_anat_requirements(args):
    """
    not all subjects have ribbon file. 
    this is common point of failure in pipeline.
    remove subject from list.
    """
    
    import s3fs
    fs = s3fs.S3FileSystem()

    return [arg for arg in args if fs.exists(f'dhcp-afq/dhcp_anat_pipeline/sub-{arg[0]}/ses-{arg[1]}/anat/sub-{arg[0]}_ses-{arg[1]}_desc-ribbon_space-T2w_dseg.nii.gz')]


# anat and dmri have different subject session pairs 558 and 490 respectively
# will only want the intesection
dhcp_anat_sub_ses = get_subject_session_pair('dhcp-afq/dhcp_anat_pipeline/')
dhcp_dmri_sub_ses = get_subject_session_pair('dhcp-afq/dhcp_dmri_pipeline/')

args = sorted(set(dhcp_anat_sub_ses) & set(dhcp_dmri_sub_ses))
args = check_anat_requirements(args) # 467 subject/session pairs
args = attach_keys(args)
print(len(args))

467


## Local

### clean up

## AWS

### CloudKnot

In [4]:
import cloudknot as ck

In [54]:
from datetime import datetime
timestamp = datetime.now().isoformat()[:-7].replace(':','-')[:-3]
timestamp

'2021-04-25T09-49'

In [6]:
# use manjari aws credentials
ck.set_profile('dbloom') 

In [7]:
# specify region
ck.set_region('us-west-2')

**NOTE:**

Assumes docker image `dhcp`

- image was built locally using `./Dockerfile` and `docker build -t dhcp .`

- reason was to update MRtrix to 3.0.2, if and when pennbbl/qsiprep includes this version of MRTrix the custom docker image will no longer be necessary.

In [8]:
dockerImage = ck.DockerImage(
    name = 'dhcp-mrtrix',
    func = dhcp_tractography_pipeline,
    base_image = 'dhcp:latest',
    overwrite = True
)

**NOTE:**

Issue with `pipreqs` generating `requirements.txt`: need to edit and replace `skimage` with `scikit-image`

In [9]:
import fileinput
import os.path as op

for line in fileinput.input(op.join(dockerImage.build_path, 'requirements.txt'), inplace = True):
    print(line.replace('skimage', 'scikit-image').rstrip())

In [10]:
dockerImage.build(tags=["dhcp-mrtrix-"+ timestamp])
repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())
dockerImage.push(repo=repo)
print(dockerImage.repo_uri)

677069060729.dkr.ecr.us-west-2.amazonaws.com/cloudknot:dhcp-mrtrix-2021-04-23T09-01


**NOTE:**

cloudknot currently only allows one bucket per profile and bucket names must be unique, therefore must manually create and set active profile.

**NOTE:**

- `retries` - 3 to reduce chance of spotinstance issues

- `bid_percentage` - 105

- `max_vcpus` - number of subjects = 490 * `job_def_vcpus`

- `volume_size` - 27GB (docker image) + 13GB (data) + 8GB (room for OS)


run remaning subject in the 4 regions

# run subset of subjects

In [ ]:
def bins(arg_list, bin_size):
    return (arg_list[i:i+bin_size] for i in range(0, len(arg_list), bin_size))

## Confirm all arguments processed

In [58]:
def check_subjects():
    import s3fs
    fs = s3fs.S3FileSystem()
    for arg in args:
        if not fs.exists(f'dhcp-afq/mrtrix/sub-{arg[0]}'):
            print(arg[0])
            if not fs.exists(f'dhcp-afq/mrtrix/sub-{arg[0]}/ses-{arg[1]}/'):
                print(arg[0], arg[1])
            
check_subjects()